# Задание 1.

Нужно обучить трансформер на этом же или на другом корпусе (можно взять другую языковую пару с того же сайте) и оценивать его на всей тестовой выборке (а не на 10 примерах как сделал я). Чтобы получить 2 доп балла вам нужно будет придумать как оптимизировать функцию translate. Подсказка: модель может предсказывать батчами.

In [1]:
!apt-get install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-25ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [2]:
!pip install tokenizers matplotlib sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post5-py3-none-any.whl size=2950 sha256=215d7e7c1f9d62c67d86b89127e65269a1c1e8c159a99ff6a5f9b2ebb5b9320e
  Stored in directory: /root/.cache/pip/wheels/38/1f/8d/4f812c590e074c1e928f5cec67bf5053b71f38e2648739403a
Successfully built sklearn


In [3]:
!pip install --upgrade jedi==0.17.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: parso
    Found existing installation: parso 0.8.3
    Uninstalling parso-0.8.3:
      Successfully uninstalled parso-0.8.3


In [4]:
!pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import tensorflow as tf
from tokenizers import BertWordPieceTokenizer

from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase
from tokenizers.trainers import WordPieceTrainer
from tokenizers import decoders

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML
import matplotlib.pyplot as plt
%matplotlib inline

Данные взяты вот отсюда - https://opus.nlpl.eu/opus-100.php (раздел с отдельными языковыми парами)

In [6]:
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.en

--2023-06-19 20:25:34--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121340806 (116M)
Saving to: ‘opus.en-ru-train.ru’

opus.en-ru-train.ru 100%[===================>] 115.72M  28.2MB/s    in 5.1s    

2023-06-19 20:25:40 (22.6 MB/s) - ‘opus.en-ru-train.ru’ saved [121340806/121340806]

--2023-06-19 20:25:40--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67760131 (65M)
Saving to: ‘opus.en-ru-train.en’

opus.en-ru-train.en 100%[===================>]  64.62M  22.0MB/s    in 2.9s    

2023-06-19 20:25:44 (22.0 MB/s) - ‘op

In [7]:
en_sents = open('opus.en-ru-train.en').read().lower().splitlines()
ru_sents = open('opus.en-ru-train.ru').read().lower().splitlines()

Пример перевода с английского на русский

In [8]:
en_sents[-1], ru_sents[-1]

('so what are you thinking?', 'ну и что ты думаешь?')

In [9]:
tokenizer_en = Tokenizer(WordPiece(), )
tokenizer_en.normalizer = normalizers.Sequence([Lowercase()])
tokenizer_en.pre_tokenizer = Whitespace()

trainer_en = WordPieceTrainer(
          vocab_size=30000, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]"])
tokenizer_en.train(files=["opus.en-ru-train.en"], trainer=trainer_en )

tokenizer_ru = Tokenizer(WordPiece(), )
tokenizer_ru.normalizer = normalizers.Sequence([Lowercase()])
tokenizer_ru.pre_tokenizer = Whitespace()

trainer_ru = WordPieceTrainer(
          vocab_size=30000, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]"])
tokenizer_ru.train(files=["opus.en-ru-train.ru"], trainer=trainer_ru )

In [10]:
' '.join(tokenizer_ru.encode('Пример текста с редким словом').tokens)

'пример текста с ред ##ким словом'

In [11]:
tokenizer_en.decoder = decoders.WordPiece()
tokenizer_ru.decoder = decoders.WordPiece()

In [12]:
tokenizer_en.save('tokenizer_en')
tokenizer_ru.save('tokenizer_ru')

In [13]:
tokenizer_en = Tokenizer.from_file("tokenizer_en")
tokenizer_ru = Tokenizer.from_file("tokenizer_ru")

In [14]:
def encode(text, tokenizer, target=False):
    return [tokenizer.token_to_id('[CLS]')] + tokenizer.encode(text).ids + [tokenizer.token_to_id('[SEP]')]


In [15]:
X_en = [encode(t, tokenizer_en) for t in en_sents]
X_ru = [encode(t, tokenizer_ru, target=True) for t in ru_sents]

In [16]:
max_len_en, max_len_ru = 20, 20

In [17]:
PAD_IDX = tokenizer_ru.token_to_id('[PAD]')
PAD_IDX

3

In [18]:
tokenizer_en.token_to_id('[PAD]')

3

In [19]:
X_en = tf.keras.preprocessing.sequence.pad_sequences(
              X_en, maxlen=max_len_en, padding='post', value=tokenizer_en.token_to_id('[PAD]'))

X_ru_out = tf.keras.preprocessing.sequence.pad_sequences(
              [x[1:] for x in X_ru], maxlen=max_len_ru-1, padding='post',
              value=tokenizer_en.token_to_id('[PAD]'))

X_ru_dec = tf.keras.preprocessing.sequence.pad_sequences(
              [x[:-1] for x in X_ru], maxlen=max_len_ru-1,
              padding='post', value=tokenizer_ru.token_to_id('[PAD]'))

In [20]:
X_en.shape, X_ru_out.shape

((1000000, 20), (1000000, 19))

In [21]:
X_en_train, X_en_valid, X_ru_dec_train, X_ru_dec_valid, X_ru_out_train, X_ru_out_valid = train_test_split(X_en,
                                                                                                      X_ru_dec,
                                                                                                      X_ru_out,
                                                                                                      test_size=0.05)

# Transformer

In [22]:
def scaled_dot_product_attention(query, key, value, mask):
    """Calculate the attention weights. """
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    # scale matmul_qk
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

    # add the mask to zero out padding tokens
    if mask is not None:
        logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k)
    attention_weights = tf.nn.softmax(logits, axis=-1)

    output = tf.matmul(attention_weights, value)

    return output

In [23]:
class MultiHeadAttention(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(
            inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        # linear layers
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # split heads
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # scaled dot-product attention
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # concatenation of heads
        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # final linear layer
        outputs = self.dense(concat_attention)

        return outputs

In [24]:
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, PAD_IDX), tf.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]

In [25]:
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

In [26]:
class PositionalEncoding(tf.keras.layers.Layer):

    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):

        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [27]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

    return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [28]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name="encoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [29]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
    attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

    attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [30]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [31]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                max_len,
                name="transformer"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

    enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)
    # mask the future tokens for decoder inputs at the 1st attention block
    look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)
    # mask the encoder outputs for the 2nd attention block
    dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

    enc_outputs = encoder(
      vocab_size=vocab_size[0],
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      max_len=max_len[0],
    )(inputs=[inputs, enc_padding_mask])

    dec_outputs = decoder(
      vocab_size=vocab_size[1],
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      max_len=max_len[1],
    )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

    outputs = tf.keras.layers.Dense(units=vocab_size[1], name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [32]:
L  = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none',)

def loss_function(y_true, y_pred):
    loss = L(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, PAD_IDX), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

In [33]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [34]:
tf.keras.backend.clear_session()

NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8
UNITS = 512
DROPOUT = 0.1



mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = transformer(
        vocab_size=(tokenizer_en.get_vocab_size(),tokenizer_ru.get_vocab_size()),
        num_layers=NUM_LAYERS,
        units=UNITS,
        d_model=D_MODEL,
        num_heads=NUM_HEADS,
        dropout=DROPOUT,
        max_len=[max_len_en, max_len_ru])

#     learning_rate = CustomSchedule(D_MODEL)

    optimizer = tf.keras.optimizers.Adam(
        0.001, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    def accuracy(y_true, y_pred):
#         y_true = tf.reshape(y_true, shape=(-1, max_len_ru - 1))
        return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)


    model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
    checkpoint = tf.keras.callbacks.ModelCheckpoint('model_ruen',
                                                monitor='val_loss',
                                                verbose=1,
                                            save_weights_only=True,
                                            save_best_only=True,
                                            mode='min',
                                            save_freq='epoch')

In [35]:
model.fit((X_en_train, X_ru_dec_train), X_ru_out_train,
             validation_data=((X_en_valid, X_ru_dec_valid), X_ru_out_valid),
             batch_size=500,
             epochs=1,
             callbacks=[checkpoint]
             )




1900/1900 [==============================] - ETA: 0s - loss: 2.5608 - accuracy: 0.2216
Epoch 1: val_loss improved from inf to 1.88213, saving model to model_ruen
1900/1900 [==============================] - 773s 395ms/step - loss: 2.5608 - accuracy: 0.2216 - val_loss: 1.8821 - val_accuracy: 0.2855


In [40]:
def translate(text):
    input_ids = encode(text.lower(), tokenizer_en)

    input_ids = tf.keras.preprocessing.sequence.pad_sequences(
                                      [input_ids], maxlen=max_len_en, padding='post')



    output_ids = [tokenizer_ru.token_to_id('[CLS]') ]

    pred = model((input_ids, tf.cast([output_ids], tf.int32)), training=False)


    while pred.numpy().argmax(2)[0][-1] not in [tokenizer_ru.token_to_id('[SEP]'),
                                                            ]:
        if len(output_ids) > max_len_ru:
            break
        output_ids.append(pred.numpy().argmax(2)[0][-1])
        pred = model((input_ids, tf.cast([output_ids], tf.int32)), training=False)

    return tokenizer_ru.decode(output_ids[1:])
    # ' '.join([tokenizer_ru.id_to_token(i) for i in output_ids[1:]])

In [37]:
def translate(texts):
    input_ids = encode(texts.lower(), tokenizer_en)

    input_ids = tf.keras.preprocessing.sequence.pad_sequences(
                                  input_ids, maxlen=max_len_en, padding='post')

    output_ids = [[tokenizer_ru.token_to_id('[CLS]')] for i in range(len(texts))]

    pred = model((input_ids, tf.cast(output_ids, tf.int32)), training=False)
    end_tokens = [tokenizer_ru.token_to_id('[SEP]')] * len(texts)

    while not all(token in end_tokens for token in pred.numpy().argmax(2)[:, -1]):
        if len(output_ids[0]) > max_len_ru:
            break
        for i in range(len(texts)):
            if end_tokens[i] not in pred.numpy().argmax(2)[i]:
                output_ids[i].append(pred.numpy().argmax(2)[i][-1])

        pred = model((input_ids, tf.cast(output_ids, tf.int32)), training=False)

    outputs = [tokenizer_ru.decode(output_id[1:]) for output_id in output_ids]
    return outputs


In [44]:
translate("can you translate this sentence?")

'вы можете перевести эту рекомендацию >>?'

In [43]:
translate("please translate this sentence into russian")

''

In [ ]:
# translate("")

# BLEU

In [45]:
en_sents_test = open('opus.en-ru-test.en').read().lower().splitlines()
ru_sents_test = open('opus.en-ru-test.ru').read().lower().splitlines()

In [50]:
from tqdm import tqdm_notebook

In [53]:
from tqdm import trange

In [55]:
translations = []

for i in range(len(ru_sents_test)):
    translations.append(translate(ru_sents_test[i]))

In [56]:
translations

['текущее время hourly 1',
 'в.',
 'не знаю, что у нас.',
 'что?',
 'г.',
 'это не так.',
 '- не волнуйся, тодди',
 'posted : 22 dec 2007, 22 : 22',
 'я подзила, что за счет.',
 '«: июль 2011, 17 : 15 : 34 »',
 'ну, в разделен.',
 'в.',
 'малага и ситероидов',
 'текущее время',
 '',
 'в конце концов, это в конце концов, в конце концов,',
 '',
 'не в отличие от бишкеки, но не так ли?',
 'это самое, что - то есть, к тому, что это имеет значение.',
 'http :// www. culture. ru / f / viewtopic. php? m2',
 'трики, что...',
 'и я не могу.',
 'выучиле.',
 'в.',
 "senten ' renten ' t could to go to go",
 'устойчивое рекомендация >> бюджетные материалы',
 '- он из - за матки.',
 'un ( 19 )',
 'как можно мгновенно получить?',
 '2. 6.',
 'и я буду, что не за это.',
 'в.',
 'в.',
 '8.',
 'начало',
 'не очень хорошо.',
 'в.',
 'из серии "#',
 '',
 '- в одном из них.',
 'будущая на вкусе, что не из них.',
 'в д.',
 'и да, учитывая, что это было, и да.',
 '- это - ка.',
 '- крот, и все время.',
 'не в

In [57]:
ru_sents_test[:10]

['только бы не вылететь.',
 'и как ты только справляешься, папа, таская эти коробки взад-вперед целый день.',
 'возможно, у нас есть небольшое преимущество в переговорах.',
 'сколько времени вы будете делать то, что ему нужно?',
 '1 апреля президент нкр бако саакян принял начальника генштаба вооруженных сил республики армения генерал-полковника юрия хачатурова.',
 'г-н приснер также упомянул, что система электронного правосудия не только позволила улучшить процесс ведения дел, но также способствует значительному упорядочению процедур.',
 '- неплохо, да.',
 'posted: 15 dec 2006, 00:07',
 'и на минутку я подумал, что за ним могут следить.',
 '«: 11 октябрь 2011, 17:15:34»']

In [59]:
import nltk

In [60]:
bleus = []

for i, t in enumerate(translations):
  reference = tokenizer_ru.encode(t).tokens
  hypothesis = tokenizer_ru.encode(ru_sents_test[i]).tokens

bleus.append(nltk.translate.bleu_score.sentence_bleu([reference], hypothesis,  ))

In [61]:
(sum(bleus)/len(bleus))*100

30.29928206533524

#Задание 2.

Прочитайте главу про машинный перевод у Журафски и Маннига - https://web.stanford.edu/~jurafsky/slp3/10.pdf Ответьте своими словами в чем заключается техника back translation? Для чего она применяется и что позволяет получить? Опишите по шагам как его применить к паре en-ru на данных из семинара

Back translation (обратный перевод) - это метод, который помогает увеличить объем данных для перевода, особенно полезный в случае малоресурсных языков.

Его суть заключается в использовании модели, которая обучается на параллельных данных для перевода с target языка на source язык. Затем эта модель используется для перевода большого объема текстов на target языке. Полученные данные добавляются к имеющимся данным на source языке, и на этих расширенных данных обучается новая модель для перевода с source языка на target.

Этот метод особенно результативен, когда данных на target языке гораздо больше, чем на source языке.

Допустим, мы хотим применить этот метод к переводу с немецкого на английский язык. Для этого мы обучаем модель на параллельных данных для перевода с английского на немецкий. Затем применяем эту модель к англоязычным текстам, которых нет в параллельном датасете, и добавляем полученные данные к имеющимся. Наконец, на этих расширенных данных мы обучаем новую модель для перевода с немецкого на английский язык.